In [ ]:
import requests
import pandas as pd
import os
from datetime import datetime, timedelta

# --- CONFIGURATION ---
PROMETHEUS_URL = 'http://10.35.29.108:30900/api/v1/query_range'
PROMETHEUS_QUERY_URL = 'http://10.35.29.108:30900/api/v1/query' 

save_path = os.path.expanduser('~/Desktop')
CSV_FILENAME = os.path.join(save_path, 'kubernetes_thesis_dataset_final_batch2.csv')

# ช่วงเวลา (ย้อนหลัง 1 วัน)
END_TIME = datetime.now()
START_TIME = END_TIME - timedelta(days=3)
STEP = '1m'

# --- 1. MAPPING STEP: Instance IP -> Node Name ---
def get_node_mapping():
    print("🗺️ Building Instance-to-Node Map...")
    try:
        # kube_node_info มีทั้ง label 'node' และ 'instance'
        response = requests.get(PROMETHEUS_QUERY_URL, params={'query': 'kube_node_info'})
        data = response.json()
        mapping = {}
        if data['status'] == 'success':
            for result in data['data']['result']:
                metric = result['metric']
                if 'node' in metric and 'instance' in metric:
                    # Map instance (IP:Port) -> node name
                    # บางที instance มี port ติดมา ต้องระวัง
                    mapping[metric['instance']] = metric['node']
        print(f"   ✅ Found mappings: {mapping}")
        return mapping
    except Exception as e:
        print(f"   ⚠️ Mapping failed: {e}")
        return {}

# --- METRICS DEFINITION ---
QUERIES = {
    # -- Usage (ใช้โดย instance) --
    # Mapping เปลี่ยนชื่อ column ทีหลัง
    'node_cpu_usage': {
        'query': 'sum(rate(container_cpu_usage_seconds_total[1m])) by (instance)',
        'key_type': 'instance' 
    },
    'node_mem_usage': {
        'query': 'sum(container_memory_working_set_bytes) by (instance)',
        'key_type': 'instance'
    },
    
    # -- Requests/Capacity (ใช้โดย node) --
    'node_cpu_req': {
        'query': 'sum(kube_pod_container_resource_requests{resource="cpu"}) by (node)',
        'key_type': 'node'
    },
    'node_mem_req': {
        'query': 'sum(kube_pod_container_resource_requests{resource="memory"}) by (node)',
        'key_type': 'node'
    },
    'node_cpu_cap': {
        'query': 'sum(kube_node_status_allocatable{resource="cpu"}) by (node)',
        'key_type': 'node'
    },
    'node_mem_cap': {
        'query': 'sum(kube_node_status_allocatable{resource="memory"}) by (node)',
        'key_type': 'node'
    },
    
    # -- Cluster Wide --
    'cluster_pods_pending': {
        'query': 'sum(kube_pod_status_phase{phase="Pending"})',
        'key_type': 'cluster'
    }
}

def fetch_metric(name, config, node_map):
    query = config['query']
    key_type = config['key_type']
    
    print(f"Fetching {name}...")
    params = {
        'query': query,
        'start': START_TIME.timestamp(),
        'end': END_TIME.timestamp(),
        'step': STEP
    }
    
    try:
        response = requests.get(PROMETHEUS_URL, params=params)
        data = response.json()
        
        if data['status'] == 'success' and data['data']['result']:
            frames = []
            for result in data['data']['result']:
                # หาชื่อ Key (Node name)
                metric = result['metric']
                raw_key = metric.get(key_type) or 'unknown'
                
                # ถ้าเป็น instance ให้แปลงเป็น node name
                if key_type == 'instance':
                    # ลองหาใน Map ตรงๆ
                    node_name = node_map.get(raw_key)
                    # ถ้าไม่เจอ ลองตัด Port ออก (เช่น 10.0.0.1:10250 -> 10.0.0.1)
                    if not node_name and ':' in raw_key:
                        base_ip = raw_key.split(':')[0]
                        # ลองค้นใน Map อีกที (เผื่อ Map เก็บแต่ IP)
                        # (ส่วนนี้ทำเผื่อไว้ simple match)
                        for k, v in node_map.items():
                            if base_ip in k:
                                node_name = v
                                break
                    
                    final_key = node_name if node_name else raw_key # ถ้าหาไม่เจอจริงๆ ใช้ค่าเดิม
                elif key_type == 'cluster':
                    final_key = 'total'
                else:
                    final_key = raw_key

                # สร้าง DataFrame
                values = result['values']
                df = pd.DataFrame(values, columns=['timestamp', 'value'])
                df['timestamp'] = pd.to_numeric(df['timestamp'])
                df['value'] = pd.to_numeric(df['value'])
                df = df.set_index('timestamp')
                
                # ตั้งชื่อ Column: metric_nodeName
                col_name = f"{name}_{final_key}" if final_key != 'total' else name
                df = df.rename(columns={'value': col_name})
                frames.append(df)
            
            if frames:
                return pd.concat(frames, axis=1)
                
        return None
    except Exception as e:
        print(f"Error {name}: {e}")
        return None

def main():
    print(f"--- Starting Final Data Extraction ---")
    
    # 1. Build Map
    node_map = get_node_mapping()
    
    final_df = None
    
    # 2. Fetch All Metrics
    for name, config in QUERIES.items():
        df = fetch_metric(name, config, node_map)
        if df is not None:
            # Join
            df = df.loc[~df.index.duplicated(keep='first')] # กันเหนียว
            if final_df is None:
                final_df = df
            else:
                final_df = final_df.join(df, how='outer')
    
    # 3. Save
    if final_df is not None:
        final_df = final_df.fillna(0)
        final_df.index = pd.to_datetime(final_df.index, unit='s')
        final_df.to_csv(CSV_FILENAME)
        print(f"\n✅ Success! Data saved to {CSV_FILENAME}")
        print(f"Rows: {len(final_df)}")
        print(f"Columns: {final_df.columns.tolist()}")
        print(final_df.head())
    else:
        print("\n❌ Failed.")

if __name__ == "__main__":
    main()